In this notebook we perform l1-regularization with different regularization strengths and compare it.
L1-regularization assumes a laplacian prior on the parameter with location $\mu=0$ and scale $b=1/\lambda$, where $\lambda$ is called the penalization strenght. objective function takes then the form
$$
J_{l_1}(\theta, \lambda)=-\log(\mathcal{p}(\mathcal{D}|\theta))+\lambda\sum_{i=1}^{n_{\theta}}|\theta_i|,
$$
where the first part is the negative log-likelihood (the normal objective function) and the second part the penality term. 
We perform the regularization in a way that we use the optimal parameters from the optimization of the regularized objective function with strength $\lambda_{k-1}$ as starting points for the optimzation with strength $\lambda_k$.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import petab
import pypesto
import pypesto.petab

import l1_regularization
import l1_regularization_visualization


In [ ]:
model_name= "Spoegler_conversionReaction"
param_scale= "lin"
n_sigma = 1


# the yaml file includes all important links to other files
yaml_config ="petab/"+param_scale+"/"+model_name+".yaml"
# create a petab problem
petab_problem = petab.Problem.from_yaml(yaml_config)

# or create from folder (did not work)
# petab_problem = petab.Problem.from_folder("petab/"+param_scale)

# import to amici
importer = pypesto.petab.PetabImporter(petab_problem)
importer.compile_model()

model = importer.create_model()

lin_objective = importer.create_objective()

In [3]:
# initialize l1-regularization

lambda_range = [0,100]
n_lambda = 100

regularization = l1_regularization.l1_regularization(model = model,
                                                    petab_problem = petab_problem,
                                                    linear_objective = lin_objective,
                                                    param_scale = param_scale,
                                                    lambda_range = lambda_range,
                                                    n_lambda = n_lambda
                                                    )

In [ ]:
# perform regularized estimation 
n_starts = 100 

regularized_results = regularization(n_starts) 

In [ ]:
reg_path = l1_regularization.compute_regularization_path(result_list = regularized_results,
                                       lambda_range = lambda_range,
                                       n_lambda = n_lambda, 
                                       n_sigma = n_sigma)

In [ ]:
lambda_range_log = [np.log10(lambda_range[0]), np.log10(lambda_range[1])]
lambdas = -np.linspace(lambda_range_log[0], lambda_range_log[1], n_lambda)

par_names = ['$\\theta_1$', '$\\theta_2$', '$\\theta_3$']

In [ ]:
# plot regularization path
l1_regularization_visualization.plot_regularization_path(reg_path=reg_path,
                                                     lambdas=lambdas,
                                                     parameter_names=par_names,
                                                     n_sigma=n_sigma)

In [ ]:
# plot bars

# define threshold which parameters are set to 0
threshold_linear = 1e-8

fig = plt.figure()
ax = fig.add_subplot(111)

l1_regularization_visualization.plot_regularization_bars(reg_path=reg_path,
                                                     lambdas=lambdas,
                                                     threshold=threshold_linear,
                                                     par_names=par_names,
                                                     ax=ax,
                                                     opt_lambda=None,
                                                     v_min=0,
                                                     v_max=1,
                                                     cb_ticks=None)

plt.show()

In [ ]:
# plot number of parameters

fig = plt.figure()
ax = fig.add_subplot(111)
l1_regularization_visualization.plot_number_of_parameters(reg_path=reg_path,
                                                      lambdas=lambdas,
                                                      threshold=threshold_linear,
                                                      ax=ax,
                                                      opt_lambda=None)
plt.show()